In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.quantum import TensorProduct

In [ ]:
kx_sym, ky_sym = sp.symbols('k_x k_y', real = True)
alpha_sym = sp.symbols('alpha', real = True, positive = True)
beta_sym = sp.symbols('beta', real = True, positive = True)
gamma_z, lambda_z = sp.symbols('gamma_z lambda_z', real = True)

In [ ]:
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

In [ ]:
h_1d =  (sp.sin(kx_sym) + alpha_sym * sp.sin(2*kx_sym)) * sx
h_1d += (sp.cos(kx_sym) + beta_sym  * sp.cos(2*kx_sym)) * sz

In [ ]:
H_layered = sp.Matrix(np.zeros((4,4)))
H_layered += TensorProduct(sz,h_1d)
H_layered += TensorProduct(sy,s0) * lambda_z * sp.sin(ky_sym)
H_layered += TensorProduct(sx,s0) * (gamma_z + lambda_z * sp.cos(ky_sym))

In [ ]:
H_layered

In [ ]:
Nx = 100
Ny = 30
Nbands = 4
Nocc = 2

params = {}
params["Nx"] = Nx
params["Ny"] = Ny
params["Nbands"] = Nbands
params["Nocc"] = Nocc

In [ ]:
Kxs = np.linspace(0, 2*np.pi, Nx, endpoint = False)
Kys = np.linspace(0, 2*np.pi, Ny, endpoint = False)

In [ ]:
H_hodti_fixalpha = H_layered.subs({alpha_sym : 2.0, beta_sym: 2.0, gamma_z: 0.5, lambda_z: 1})

In [ ]:
hfunc = sp.lambdify((kx_sym,ky_sym), H_hodti_fixalpha, modules = "numpy")

In [ ]:
eigenvalues = np.zeros((Nx,Ny,Nbands))
eigenstates = np.zeros((Nx,Ny,Nbands,Nbands), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        vals, vecs = np.linalg.eigh(hfunc(Kxs[i],Kys[j]))

        ind = np.argsort(vals)

        eigenvalues[i,j] = vals[ind]
        eigenstates[i,j] = vecs[:,ind]

In [ ]:
links_y_occ = np.zeros((Nx,Ny,Nocc,Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        overlap_occ = eigenstates[i, (j+1)%Ny, :, :Nocc].conj().T @ eigenstates[i, j, :, :Nocc]

        U_occ, _, Vh_occ = np.linalg.svd(overlap_occ)

        links_y_occ[i, j, :, :] = U_occ @ Vh_occ

In [ ]:
Wilsonloops_y_occ = np.zeros((Nx, Nocc, Nocc)).astype(np.complex128)

for i in range(Nx):
    W_occ = np.eye(Nocc).astype(np.complex128)

    for j in range(Ny):
        W_occ = links_y_occ[i, j, :, :] @ W_occ

    Wilsonloops_y_occ[i] = W_occ

In [ ]:
nu_vals_y = np.zeros((Nx, Nocc))
nu_vecs_y = np.zeros((Nx, Nocc, Nocc)).astype(np.complex128)

for i in range(Nx):
    evals, evecs = np.linalg.eig(Wilsonloops_y_occ[i, :, :])
    angles = np.angle(evals)/(2*np.pi)

    idx = np.argsort(angles)
    angles = angles[idx]
    evecs = evecs[:, idx]

    nu_vals_y[i, :] = angles[:]
    nu_vecs_y[i, :, :] = evecs[:, :]

In [ ]:
fig = plt.figure(figsize = (4,3))
plt.plot(Kxs,nu_vals_y[:,0], 'k-', alpha = 0.7)
plt.plot(Kxs,nu_vals_y[:,1], 'k-', alpha = 0.7)

plt.xlabel(r"$k_x$", fontsize = 16, labelpad = -2)
plt.xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], [r"$0$", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"])
plt.ylabel(r"$\nu_y$", fontsize = 16, labelpad = -3)
plt.tight_layout()
plt.show()

In [ ]:
wb = np.zeros((Nx, 4)).astype(np.complex128)

for i in range(Nx): 
    wb[i,:] = eigenstates[i,0,:,0] * nu_vecs_y[i,0,0] + eigenstates[i,0,:,1] * nu_vecs_y[i,1,0]

In [ ]:
Nx2 = int(Nx/2)
halfBZ_vecs = wb[:Nx2,:]

W = 1 
for i in range(Nx2):
    ol = np.dot(halfBZ_vecs[(i+1)%Nx2,:].conj().T, halfBZ_vecs[i,:])
    W *= ol / np.abs(ol)
print(W)
print(np.angle(W)/(2*np.pi))